In [1]:
from datetime import date, datetime
import ipywidgets as widgets
from IPython.display import display,clear_output
import math
import ROOT
import numpy as np, pandas as pd
import bokeh
import yaml
import base64
from bokeh.io import output_notebook, show, curdoc
from bokeh.models import CustomJS, MultiSelect, Button, ColumnDataSource, DateRangeSlider, CheckboxButtonGroup, FileInput
from bokeh.events import ButtonClick, MenuItemClick
from bokeh.layouts import row, column
output_notebook()

Welcome to JupyROOT 6.24/06


Loading BokehJS ...

### Open the TFile containing the downloaded objects

In [2]:
rootDataFile = ROOT.TFile.Open("../../Data/UserFiles/visClustersData.root","READ")

with open('../../Data/UserFiles/directoryTree.txt') as directoryTree: 
    lines = directoryTree.readlines()
    
files = list()
paths = list()
time_stamps = list()
dates = list()
types = list()
numbers = list()

active_file = ''
file_type = ''
file_number = 1
for line in lines:
    if line[0:7] == 'qc/TPC/':
        active_file = line[17:-1]
        files.append(active_file)
        file_number = 1
    elif line[0:5] == 'Type:':
        file_type = line[6:-1]
    elif line[0] == '-':
        paths.append(active_file)
        types.append(file_type)
        numbers.append(file_number)
        file_number += 1
        time_stamp = int(line[3:-1])
        time_stamps.append(time_stamp)
        date = datetime.utcfromtimestamp(math.ceil(time_stamp)/1000).strftime('%Y-%m-%d-%H:%M:%S')
        dates.append(date)
    
files_data = pd.DataFrame({'Path': paths, 'Type': types, 'TimeStamp': time_stamps, 'Date':dates, 'Number': numbers})
type_set = sorted(list(set(types)))

In [3]:
files_data

,Path,Type,TimeStamp,Date,Number
0,h2DEtaPhi,TH2F,1637847917847,2021-11-25-13:45:17,1
1,h2DEtaPhi,TH2F,1624365699489,2021-06-22-12:41:39,2
2,h2DEtaPhi,TH2F,1610617812666,2021-01-14-09:50:12,3
3,h2DEtaPhi,TH2F,1635511476391,2021-10-29-12:44:36,4
4,h2DEtaPhi,TH2F,1624279264994,2021-06-21-12:41:04,5
...,...,...,...,...,...
1398,hSign,TH1F,1624536124137,2021-06-24-12:02:04,77
1399,hSign,TH1F,1624612022186,2021-06-25-09:07:02,78
1400,hSign,TH1F,1624621552534,2021-06-25-11:45:52,79
1401,hSign,TH1F,1625243148262,2021-07-02-16:25:48,80


### Create the dashboard component for browsing files to download

In [4]:
%jsroot on

selected_data = []
selected_files = files_data.copy(deep=True)
selected_types = type_set
time_range = [files_data['TimeStamp'].min(),files_data['TimeStamp'].max()]
shown_files = files

def file_browser(doc):
    def update_button_label():
        label = "{} files selected".format(str(len(selected_files.index)))
        file_select.title = label
        
    def update_file_select_list(shown_files):
        new_list = list(set(shown_files['Path']))
        file_select.options = new_list
        file_select.value = []
        update_time_range(shown_files)
        
    def update_time_range(shown_files):
        if not shown_files.empty:
            date_range.value=[shown_files['TimeStamp'].min(),shown_files['TimeStamp'].max()]
            date_range.start=shown_files['TimeStamp'].min()
            date_range.end=shown_files['TimeStamp'].max()
        
    def select_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types
        selected_data = file_select.value
        time_condition = ((files_data['TimeStamp'] >= time_range[0]) & (files_data['TimeStamp'] <= time_range[1]))
        path_condition = files_data['Path'].map(lambda x: x in selected_data)
        type_condition = files_data['Type'].map(lambda x: x in selected_types)
        selected_files = files_data[path_condition & time_condition & type_condition]
        update_button_label()
        
    def date_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types
        time_range = date_range.value
        time_condition = ((files_data['TimeStamp'] >= time_range[0]) & (files_data['TimeStamp'] <= time_range[1]))
        path_condition = files_data['Path'].map(lambda x: x in selected_data)
        type_condition = files_data['Type'].map(lambda x: x in selected_types)
        selected_files = files_data[path_condition & time_condition & type_condition]
        update_button_label()
        
    def type_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types, shown_files
        selected_types = [type_set[i] for i in checkbox.active]
        time_condition = ((files_data['TimeStamp'] >= time_range[0]) & (files_data['TimeStamp'] <= time_range[1]))
        path_condition = files_data['Path'].map(lambda x: x in selected_data)
        type_condition = files_data['Type'].map(lambda x: x in selected_types)
        selected_files = files_data[path_condition & time_condition & type_condition]
        shown_files = files_data[type_condition]
        update_file_select_list(shown_files)
        update_button_label()
        
    def button_callback(event):
        global selected_data, selected_files, files_data, time_range, selected_types, canvas_no
        to_plot.Draw()
    
    checkbox = CheckboxButtonGroup(labels=type_set,active=list(range(0,len(type_set))))
    checkbox.on_change("active",type_callback)
        
    date_range = DateRangeSlider(value=[files_data['TimeStamp'].min(),files_data['TimeStamp'].max()],
                                    start=files_data['TimeStamp'].min(), end=files_data['TimeStamp'].max())
    date_range.on_change("value_throttled",date_callback)    
    
    file_select = MultiSelect(title="Files", value=[], options=files, height=200)
    file_select.on_change("value", select_callback)

    #button = Button(label="Show", button_type="success") 
    #button.on_event(ButtonClick, button_callback)
    doc.add_root(column(file_select,checkbox,date_range))
show(file_browser)

button = widgets.Button(description="Show")
output = widgets.Output()
display(button, output)
def on_button_clicked(b):
    with output:
        global canvas, to_plot
        canvas = ROOT.TCanvas()
        for i in range(1,(selected_files.Path).size):
            root_name = selected_files.Path.iloc[i]+";"+str(selected_files.Number.iloc[i])
            to_plot = rootDataFile.Get(root_name)
            to_plot.Draw("same,colz")
        canvas.Draw()

button.on_click(on_button_clicked)

Button(description='Show', style=ButtonStyle())

Output()